In [1]:
import os
import sys
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from sklearn.decomposition import PCA
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

28-Sep-22 11:58:23 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
config = general.load_config_file("/allen/aics/assay-dev/MicroscopyOtherData/Viana/datasets/variance/")

In [3]:
config["project"]["local_staging"] = "/allen/aics/assay-dev/MicroscopyOtherData/Viana/datasets/variance/"

In [5]:
control = controller.Controller(config)
device = io.LocalStagingIO(control)

In [6]:
df = device.load_step_manifest("preprocessing")

In [7]:
space = shapespace.ShapeSpace(control)
space.execute(df)

In [ ]:
# space.set_active_shape_mode("NUC_MEM_PC1", digitize=True)
# mpId = control.get_center_map_point_index()
# space.set_active_map_point_index(mpId)
# CellIds = space.get_active_cellids()

In [8]:
space.set_active_shape_mode("NUC_MEM_PC1", digitize=True)
df_withbins = df.join(space.meta[space.meta["mpId"] == control.get_center_map_point_index()]["mpId"])
df_withbins = df_withbins[df_withbins["mpId"].notna()]

In [27]:
SEED = 24
sampled_strucs = []
for _, data in df_withbins.groupby(["structure_name"]):
    sampled_struc_data = data.sample(32, random_state=SEED)
    sampled_strucs.append(sampled_struc_data)
df_subset = pd.concat(sampled_strucs)

In [28]:
df_subset["structure_name"].value_counts()

AAVS1        32
NPM1         32
TOMM20       32
TJP1         32
ST6GAL1      32
SON          32
SMC1A        32
SLC25A17     32
SEC61B       32
RAB5A        32
PXN          32
NUP153       32
MYH10        32
ACTB         32
LMNB1        32
LAMP1        32
HIST1H2BJ    32
GJA1         32
FBL          32
DSP          32
CTNNB1       32
CETN2        32
ATP2A2       32
ACTN1        32
TUBA1B       32
Name: structure_name, dtype: int64

In [30]:
config

{'appName': 'cvapipe_analysis',
 'project': {'local_staging': '/allen/aics/assay-dev/MicroscopyOtherData/Viana/datasets/variance/',
  'overwrite': False},
 'data': {'cell': {'alias': 'MEM',
   'channel': 'membrane_segmentation',
   'color': '#F200FF'},
  'cell-roof': {'alias': 'MEMROOF',
   'channel': 'membrane_segmentation_roof',
   'color': '#F200FF'},
  'nucleus': {'alias': 'NUC',
   'channel': 'dna_segmentation',
   'color': '#3AADA7'},
  'structure': {'alias': 'STR',
   'channel': 'struct_segmentation_roof',
   'color': '#000000'},
  'raw-structure-str': {'alias': 'RAWSTR_MASKEDBY_STR',
   'channel': 'structure',
   'color': '#000000'},
  'raw-structure-nuc': {'alias': 'RAWSTR_MASKEDBY_NUC',
   'channel': 'structure',
   'color': '#000000'},
  'raw-structure-mem': {'alias': 'RAWSTR_MASKEDBY_MEM',
   'channel': 'structure',
   'color': '#000000'}},
 'features': {'aliases': ['NUC',
   'MEM',
   'STR',
   'RAWSTR_MASKEDBY_STR',
   'RAWSTR_MASKEDBY_NUC',
   'RAWSTR_MASKEDBY_MEM'],
  '

In [31]:
from cvapipe_analysis.steps.parameterization.parameterization_tools import Parameterizer

parameterizer = Parameterizer(control)
parameterizer.subfolder = "parameterization/representations_reconerr_seg"

In [ ]:
# using raw data

In [ ]:
# using seg data